In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys

from dotenv import load_dotenv

load_dotenv("../config/prod.env")

import yaml
with open("../config/config.yaml") as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

import pandas as pd
import mlflow

from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Data,
    CodeConfiguration,
    Environment,
    ProbeSettings
)

from datetime import datetime


# enter details of your AML workspace
subscription_id = os.getenv("subscription_id")
resource_group = os.getenv("resource_group")
workspace = os.getenv("workspace")

credential = InteractiveBrowserCredential() #DefaultAzureCredential()
# get a handle to the workspace
ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

# Create Online Endpoint

In [ ]:
endpoint_name = cfg["online_endpoint"]["name"]

# # Get the current time
# current_time = datetime.now()
# # Convert the current time to a string using the strftime method
# time_string = current_time.strftime("%Y%m%d%H%M%S")
# endpoint_name = f"{endpoint_name}-{time_string}"

print(f"Endpoint name: {endpoint_name}")

In [ ]:
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name, #cfg["endpoint"]["name"],
    description=cfg["online_endpoint"]["description"],
    auth_mode=cfg["online_endpoint"]["auth_mode"],
)

## Test Local Endpoint

In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint, local=True)


In [ ]:
model = Model(path="../models/model")
env = Environment(
    conda_file="../environment/conda_env/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

local_deployment = ManagedOnlineDeployment(
    name=cfg["online_endpoint"]["deployment"]["name"],
    description=cfg["online_endpoint"]["deployment"]["description"],
    endpoint_name=cfg["online_endpoint"]["name"],
    model=model,
    code_configuration=CodeConfiguration(
        code="../src/", scoring_script="score.py"
    ),
    environment=env,
    instance_type=cfg["online_endpoint"]["deployment"]["instance_type"],
    instance_count=cfg["online_endpoint"]["deployment"]["instance_count"],
    # liveness_probe=ProbeSettings,
    # readiness_probe=,
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(
    deployment=local_deployment, local=True
)

In [ ]:
ml_client.online_endpoints.get(name=endpoint_name, local=True)


In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="../data/endpoint/sample_request.json",
    local=True,
)

In [ ]:
logs = ml_client.online_deployments.get_logs(
    name=cfg["online_endpoint"]["deployment"]["name"], endpoint_name=endpoint_name, local=True, lines=50
)
print(logs)

In [ ]:
ml_client.online_endpoints.begin_delete(name=endpoint_name, local=True)


## Create live deployment

In [ ]:
ml_client.begin_create_or_update(endpoint).result()


In [ ]:
deployment = ManagedOnlineDeployment(
    name=cfg["online_endpoint"]["deployment"]["name"],
    description=cfg["online_endpoint"]["deployment"]["description"],
    endpoint_name=cfg["online_endpoint"]["name"],
    model=ml_client.models.get(name=cfg["model"]["name"], label="latest"),
    code_configuration=CodeConfiguration(
        code="../src/", scoring_script="score.py"
    ),
    environment=ml_client.environments.get(name=cfg["online_endpoint"]["environment"]["name"], version=cfg["online_endpoint"]["environment"]["version"]),
    instance_type=cfg["online_endpoint"]["deployment"]["instance_type"],
    instance_count=cfg["online_endpoint"]["deployment"]["instance_count"],
    liveness_probe=ProbeSettings(timeout=2),
    readiness_probe=ProbeSettings(timeout=2),
)

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()


In [ ]:
logs = ml_client.online_deployments.get_logs(
    name=cfg["online_endpoint"]["deployment"]["name"],
    endpoint_name=cfg["online_endpoint"]["name"], 
    lines=100, 
    container_type="storage-initializer"
)
print(logs)

In [ ]:
logs = ml_client.online_deployments.get_logs(
    name=cfg["online_endpoint"]["deployment"]["name"],
    endpoint_name=cfg["online_endpoint"]["name"], 
    lines=100, 
)
print(logs)

In [ ]:
endpoint.traffic = {deployment.name: 100}
ml_client.begin_create_or_update(endpoint).result()

# Test invoke

In [ ]:
import ast
from io import StringIO

In [ ]:
response = ml_client.online_endpoints.invoke(
    endpoint_name=cfg["online_endpoint"]["name"],
    deployment_name=cfg["online_endpoint"]["deployment"]["name"],
    request_file="../data/endpoint/sample_request.json"
)
response

In [ ]:
import ast
pd.read_json(StringIO(ast.literal_eval(response)))

# Invoke with rest api

In [ ]:
import urllib.request
import json
import os

data = {
    "inputs": [
        [0,0,1], 
        [0.8,1,8],
        [0.9,1,1]
    ]
}

body = str.encode(json.dumps(data))

url = "https://<>.westus.inference.ml.azure.com/score"
api_key = ""
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

In [ ]:
pd.read_json(StringIO(ast.literal_eval(result.decode("utf8"))))